In [222]:
#!L
import IPython.display as ipd
from scipy.io.wavfile import read
import torch 
import sys
import os

from torch.utils.data import DataLoader
from torch.autograd import Variable
from glow import WaveGlow
import json
%enable_full_walk

import warnings
warnings.filterwarnings("ignore")

In [223]:
#!L:bash
git clone https://github.com/NVIDIA/tacotron2.git

Cloning into 'tacotron2'...
remote: Enumerating objects: 406, done.
remote: Total 406 (delta 0), reused 0 (delta 0), pack-reused 406
Receiving objects: 100% (406/406), 2.69 MiB | 5.71 MiB/s, done.
Resolving deltas: 100% (205/205), done.


In [224]:
#!L
sys.path.insert(0, 'tacotron2')
from tacotron2.layers import TacotronSTFT

In [225]:
#!L
MODELTYPE = 'RUS' # RUS
sr = 16000 if MODELTYPE == 'RUS' else 22000

In [226]:
#!L
with open('config.json') as f:
        data = f.read()
config = json.loads(data)
test_data_config = config["test_data_config"]
train_data_config = config["train_data_config"]
waveglow_config = config["waveglow_config"]

In [227]:
#!L
model = WaveGlow(**waveglow_config).cuda()
model_name = 'WaveGlow_eng' if MODELTYPE == 'ENG' else 'WaveGlow_rus'
model.load_state_dict(torch.load(model_name))
model.eval()
print('Model loaded')
stft = TacotronSTFT(filter_length=test_data_config["filter_length"],
                    hop_length=test_data_config["hop_length"],
                    win_length=test_data_config["win_length"],
                    sampling_rate=test_data_config["sampling_rate"],
                    mel_fmin=test_data_config["mel_fmin"],
                    mel_fmax=test_data_config["mel_fmax"])

Model loaded


In [ ]:
#!L
test_dir = 'test_wavs_eng' if MODELTYPE == 'ENG' else 'test_wavs_rus'

In [ ]:
#!L
def run_example(path):
    sr, audio = read(f'{test_dir}/{path}')
    audio = torch.from_numpy(audio).float()
    audio_size = audio.shape[0]

    audio = audio / 32768.0

    audio_norm = Variable(audio.unsqueeze(0), requires_grad=False)
    mel_spec = stft.mel_spectrogram(audio_norm)
    mel_spec = torch.squeeze(mel_spec, 0)
    
    with torch.no_grad():
        audio = model.infer(mel_spec.unsqueeze(0).cuda(), sigma=0.6)
    audio = audio * 32768.0
    audio = audio.squeeze().cpu().numpy().astype('int16')
    
    return audio

In [ ]:
#!L
pathes = [i for i in os.listdir(test_dir) if 'wav' in i]

In [ ]:
#!L
ipd.Audio(run_example(pathes[0]), rate=sr)

In [ ]:
#!L
ipd.Audio(run_example(pathes[1]), rate=sr)

In [ ]:
#!L
ipd.Audio(run_example(pathes[2]), rate=sr)

In [ ]:
#!L
ipd.Audio(run_example(pathes[3]), rate=sr)

In [ ]:
#!L
ipd.Audio(run_example(pathes[4]), rate=sr)

In [ ]:
#!L
ipd.Audio(run_example(pathes[5]), rate=sr)

In [ ]:
#!L
ipd.Audio(run_example(pathes[6]), rate=sr)

In [ ]:
#!L
ipd.Audio(run_example(pathes[7]), rate=sr)

In [ ]:
#!L
ipd.Audio(run_example(pathes[8]), rate=sr)

In [ ]:
#!L
ipd.Audio(run_example(pathes[9]), rate=sr)

In [ ]:
#!L
